# Simulated Sky Signal in map domain

This lesson is about simulating the input sky signal using PySM 3.

## PySM 3

If you used `PySM` in the past, you most probably used `PySM 2` from https://github.com/bthorne93/PySM_public.

`PySM 3` is a rewrite of `PySM` which offers all the same functionality and the same models of `PySM 2` but is focused on:

* improving performance using just-in-time compilation and multi-threading with `numba`
* lowering memory requirements by reworking the underlying algorithms
* improved capability of running in parallel with MPI

It is available from https://github.com/healpy/pysm, it is still missing a few models and the documentation but is already integrated into TOAST to overcame the strong performance limits of `PySM 2`.

If anyone is interested in learning more about PySM 3, check the [PySM 3 tutorial](https://github.com/zonca/pysm_tutorial), we can work through this during the hack day.

## PySMSky

The lower level TOAST class is `PySMSky`, it performs the following operations:
* initialize `PySM` with the input sky configuration
* loop through all channels and for each calls `PySM` to generate the sky emission at all frequencies in the bandpass and integrate

In [ ]:
# Load common tools for all lessons
import sys
sys.path.insert(0, "..")
from lesson_tools import (
    fake_focalplane
)

# Capture C++ output in the jupyter cells
%reload_ext wurlitzer

In [ ]:
import toast
import healpy as hp
import numpy as np

In [ ]:
env = toast.Environment.get()                         
env.set_log_level("DEBUG")

In [ ]:
focal_plane = fake_focalplane()

In [ ]:
from toast.todmap import PySMSky
PySMSky?

In [ ]:
NSIDE = 64
npix = hp.nside2npix(NSIDE)

### PySM models

You can find out details about all the models available in PySM 2 and 3 at:

https://pysm-public.readthedocs.io/en/latest/models.html

In [ ]:
pysm_sky_config = ["s1", "f1", "a1", "d1"]

In [ ]:
pysm_sky = PySMSky(comm=None,
       pixel_indices=None,
       nside=NSIDE,
       units="uK_RJ",
       pysm_sky_config=pysm_sky_config
)

In [ ]:
pysm_sky

### PySM Sky object

We can directly access the underlying `PySM.Sky` object as the `sky` attribute of the `PySMSky` object.

In [ ]:
pysm_sky.sky

In [ ]:
import pysm.units as u

In [ ]:
pysm_sky.sky.get_emission(12 * u.GHz)

In [ ]:
%matplotlib inline
## _ refers to the output of the previous cell, so this works only if you run cells in sequence
hp.mollview(_[0], cmap="coolwarm", min=-100, max=1e4)

### Execute the PySMSky object

First we need bandpasses for the channels, first element of the tuple is frequency, second element are weights, we define a top-hat of 10 points with a bandwidth of 10 GHz:

In [ ]:
bandpasses = {}
for ch in focal_plane: # loops through dict keys
    bandpasses[ch] = (np.linspace(65, 75, 10), np.ones(10))

In [ ]:
bandpasses

In [ ]:
local_maps = {}
pysm_sky.exec(local_maps, out="sky", bandpasses=bandpasses)

In [ ]:
hp.mollview(local_maps["sky_0A"][0], cmap="coolwarm", min=0, max=1e3, unit="uK_RJ")

In [ ]:
local_maps["sky_0B"][0]-local_maps["sky_0A"][0]

In [ ]:
bandpasses["0B"] = (np.linspace(63, 73, 10), np.ones(10))

In [ ]:
local_maps = {}
pysm_sky.exec(local_maps, out="sky", bandpasses=bandpasses)

In [ ]:
hp.mollview(local_maps["sky_0A"][0]-local_maps["sky_0B"][0], cmap="coolwarm", unit="uK_RJ")

In [ ]:
hp.gnomview(local_maps["sky_0A"][0]-local_maps["sky_0B"][0], rot=(0,0), xsize=5000, ysize=2000, cmap="coolwarm")

## Exercise

Create a `PySMSky` object with pure CMB (model `c1`), in `uK_CMB`

Get the CMB signal for 2 channels at 2 different frequencies and verify that the maps are the same to double precision error (~1e-14).
To specify a channel at a single frequency instead of a bandpass, you can specify:

    bandpasses={"ch0":(100,1)}
    
Where the first element of the tuple is frequency in `GHz`, the second its weight.